In [2]:
### movement descriptor

###########################
## weight, th 정해야함 
## 단지 평균이 아니라 다른 처리해줘야 함
##########################


## Dense Optical Flow in OpenCV
#오른쪽으로 움직이는 물체는 빨간계열, 왼쪽으로 움직이는 물체는 녹색계열, 빠르게 움직일수록 진한색

import cv2
import numpy as np
import os
import math
from PIL import Image


#path_ = "한 영상에 대한 컷단위들 모아놓은 폴더"
path_ = '/Users/leedagyeong/Desktop/SW/SW-Maestro/2. Movie to Image/video/팽귄 cut'
cut_len = len(os.listdir(path_))-1 #총 컷 갯수

#프레임 사이즈
#나중에 폭력 vs 비폭력 분류에 feature수와 연관
row_len = 720 
col_len = 1280

row_sub = int(row_len/15) #프레임 세로/15
col_sub = int(col_len/15) #프레임 가로/15
#row_sub * col_sub이 featrue의 수가 될 것


##최종적으로 각 컷마다 Motion Binary Map 저장
MBM_means = np.zeros((row_sub,col_sub,cut_len))
#ex) MBM_means[:,:,0] : 첫번째 컷의 움직임 서술자

for cut_ith in range(cut_len):

    #path = '/Users/leedagyeong/SW-Maestro/2. Movie to Image/video/팽귄 cut/5'
    
    #cut_ith번째 컷이 들어있는 파일의 경로
    path = path_+'/'+str(cut_ith+1)
    
    #그 파일 안에 들어있는 프레임들 리스트
    frame_list = os.listdir(path)
    #프레임 갯수
    list_len = len(frame_list)-1 

    #해당 컷의 프레임들 다 모으기
    frames = np.zeros((row_len,col_len,list_len))

    for i in range(list_len):
        #frames[:,:,i] = cv2.imread(path+'/'+frame_list[i+1], cv2.IMREAD_GRAYSCALE)
        frame1 = Image.open(path+'/'+frame_list[i+1]) #프레임 불러오기
        frame_reshape = frame1.resize((row_len, col_len)) #프레임 사이즈 조정
        
        frame_reshape_ = np.array(frame_reshape)
        frames[:,:,i] = cv2.cvtColor(frame_reshape_,cv2.COLOR_BGR2GRAY) #회색으로 바꿔서 저장
        
    #각각 프레임들의 flow(11flow : 358x640x2x11)
    flows = np.zeros((row_len,col_len,2,list_len-1))

    for i in range(list_len-1):
        flows[:,:,:,i] = cv2.calcOpticalFlowFarneback(frames[:,:,i],frames[:,:,i+1], None, 0.5, 3, 15, 3, 5, 1.2, 0) 

    #flow에 따른 mag, ang (11장 : 358x640x11)
    mags = np.zeros((row_len,col_len,list_len-1))
    angs = np.zeros((row_len,col_len,list_len-1))

    for i in range(list_len-1):
        mags[:,:,i], angs[:,:,i] = cv2.cartToPolar(flows[:,:,0,i], flows[:,:,1,i])

    #이전 프레임과의 크기 및 방향의 변화량 : Ds (23x42x10 : 15x15인 서브영역때문에 358/15=23, 640/15=42)
    Ds = np.zeros((int(row_len/15),int(col_len/15),list_len-2))

    #15x15인 서브영역
    sub_mags = np.zeros((15,15,list_len-1))
    sub_angs = np.zeros((15,15,list_len-1))

    #이전 프레임과의 크기 및 방향의 변화량
    w = 0.4 #weight

    for a in range(row_sub):
        for b in range(col_sub):
            for i in range(list_len-1):
                sub_mags[:,:,i] = mags[15*a:15*(a+1),15*b:15*(b+1),i] 
                sub_angs[:,:,i] = angs[15*a:15*(a+1),15*b:15*(b+1),i]

            M = sub_mags[7,7,:]
            V = sub_angs[7,7,:]

            for j in range(len(M)-1):
                Ds[a,b,j] = (w*abs(M[j+1]-M[j]))+((1-w)*math.acos(V[j+1]*V[j]/abs(V[j+1]*V[j])))/math.radians(180) 
            Ds[np.isnan(DS)]=0


    #Motion Binary Map : 움직임 변화가 큰 부분 추출

    th = 0 #Ds가 th보다 큰 부분만

    MBM = np.zeros_like(Ds)
    MBM[Ds>th]=1


    #컷단위 평균
    MBM_mean = MBM.mean(axis=2)

    MBM_means[:,:,cut_ith] = MBM_mean


    
#최종적으로 한 영상에 대한 움직임 서술자 
#단지 평균이 아니라 다른처리해야할듯 
video_MBM_means = MBM_means.mean(axis = 2)


#video_MBM_means를 한줄로 핀게 한 영상에 대한 폭력 vs 비폭력 분류의 feature가 될것


########################################################################################
#import csv    
#f = open('output2.csv', 'w', encoding='utf-8', newline='')
#
#wr = csv.writer(f)
#wr.writerow(list(video_MBM_means.reshape(video_MBM_means.shape[0]*video_MBM_means.shape[1])))
#
#f.close()
########################################################################################

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:102: RuntimeWarning: invalid value encountered in greater
